# Merge and Clean Datasets

In [2]:
import pandas as pd
from fuzzywuzzy import process

/opt/anaconda3/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Load Data

In [4]:
# Load datasets
qs_data = pd.read_csv('qs_data.csv')
scorecard_data = pd.read_csv('scorecard_data.csv')
ntu_data = pd.read_csv('ntu_research_ranking.csv')
size_data = pd.read_csv('campus_data.csv')
sentiment_data = pd.read_csv('RMP_Data_Clean_Organized.csv')

# Clean Data

In [6]:
# Filter QS data for useful metrics only
qs_data = qs_data[['institution',
                   'ar score',
                   'er score',
                   'isr score',
                   'fsr score',
                   'ifr score']]

In [7]:
# Set standard university names
standard_names = ['Princeton University',
 'Massachusetts Institute of Technology',
 'Harvard University',
 'Stanford University',
 'Yale University',
 'California Institute of Technology',
 'Duke University',
 'Johns Hopkins University',
 'Northwestern University',
 'University of Pennsylvania',
 'Cornell University',
 'University of Chicago',
 'Brown University',
 'Columbia University',
 'Dartmouth College',
 'University of California, Los Angeles',
 'University of California, Berkeley',
 'Rice University',
 'University of Notre Dame',
 'Vanderbilt University',
 'Carnegie Mellon University',
 'University of Michigan, Ann Arbor',
 'Washington University',
 'Emory University',
 'Georgetown University',
 'University of Virginia',
 'University of North Carolina, Chapel Hill',
 'University of Southern California',
 'University of California, San Diego',
 'New York University',
 'University of Florida',
 'University of Texas, Austin',
 'Georgia Institute of Technology',
 'University of California, Davis',
 'University of California, Irvine',
 'University of Illinois, Urbana-Champaign',
 'Boston College',
 'Tufts University',
 'University of California, Santa Barbara',
 'University of Wisconsin, Madison',
 'Boston University',
 'Ohio State University, Columbus',
 'Rutgers University, New Brunswick',
 'University of Maryland, College Park',
 'University of Rochester',
 'Lehigh University',
 'Purdue University',
 'University of Georgia',
 'University of Washington',
 'Wake Forest University']

In [8]:
def standardize_column_values(df, column, standard_names, threshold=80):

    updated_values = []
    
    for value in df[column]:
        match, score = process.extractOne(value, standard_names)
        if score >= threshold:
            updated_values.append(match)  # Use the matched standard name
        else:
            updated_values.append(value)  # Keep the original name if no good match
    
    df[column] = updated_values
    return df

In [9]:
# Standardize university names across datasets
ntu_data = standardize_column_values(ntu_data, 'University Name', standard_names, threshold=80)
qs_data = standardize_column_values(qs_data, 'institution', standard_names, threshold=80)
size_data = standardize_column_values(size_data, 'College or University', standard_names, threshold=80)
scorecard_data = standardize_column_values(scorecard_data, 'University Name', standard_names, threshold=80)
sentiment_data = standardize_column_values(sentiment_data, 'University', standard_names, threshold=80)

# Rename university columns for easier merging
size_data.rename(columns={'College or University': 'University Name'}, inplace=True)
qs_data.rename(columns={'institution': 'University Name'}, inplace=True)
sentiment_data.rename(columns={'University': 'University Name'}, inplace=True)

# Rename OSU, Columbus to OSU
size_data.loc[11, 'University Name'] = 'Ohio State University, Columbus'
qs_data.loc[31, 'University Name'] = 'Ohio State University, Columbus'

In [10]:
# Merge DataFrames (outer merge for easier detection of problems)
merged_df = scorecard_data.merge(ntu_data, on='University Name', how='outer') \
                          .merge(qs_data, on='University Name', how='outer') \
                          .merge(size_data, on='University Name', how='outer') \
                          .merge(sentiment_data, on='University Name', how='outer')

# Drop duplicate universities
merged_df = merged_df.drop_duplicates(subset=['University Name']).reset_index(drop=True)

# Show DataFrame
merged_df

,University Name,In State Tuition,Out of State Tuition,Admission Rate,Number of Students,Graduation Rate 4-years,Earnings After 10 Years,Retention Rate,Diversity Score,Public or Private,...,ar score,er score,isr score,fsr score,ifr score,Acres,Safety,Facilities,Social,Average Sentiment
0,Boston College,22144,22144,0.1670,259,0.4545,62123,0.8235,0.597781,Private Nonprofit,...,20.5,12.9,18.3,19.3,12.0,240.0,4.7,4.3,3.8,0.596994
1,Boston University,62360,62360,0.1437,17668,0.8916,83238,0.9397,0.671706,Private Nonprofit,...,61.7,68.3,87.5,32.7,21.6,140.0,4.2,3.7,3.7,0.376901
2,Brown University,29720,29720,0.5664,1354,0.7078,53907,0.8445,0.431634,Private Nonprofit,...,58.3,42.4,52.0,77.8,84.5,146.0,4.5,4.4,4.5,0.694265
3,California Institute of Technology,19190,19190,0.0270,540,0.7828,53598,0.9474,0.483280,Proprietary,...,96.1,93.7,81.0,100.0,100.0,124.0,4.2,4.3,3.6,0.458000
4,Carnegie Mellon University,61344,61344,0.1130,7005,0.9214,114862,0.9730,0.673670,Private Nonprofit,...,71.2,75.3,98.2,49.1,46.6,150.0,4.3,4.0,3.2,0.324381
5,Columbia University,66139,66139,0.0395,8902,0.9528,102491,0.9792,0.679876,Private Nonprofit,...,99.6,98.3,96.8,100.0,56.1,36.0,3.5,3.9,3.3,0.356867
6,Cornell University,63200,63200,0.0790,16071,0.8800,104043,0.9600,0.661828,Private Nonprofit,...,98.1,90.5,64.0,52.9,68.7,2300.0,4.4,4.6,4.0,0.553095
7,Dartmouth College,62658,62658,0.0638,4412,0.9448,97434,0.9803,0.644170,Private Nonprofit,...,21.1,24.2,33.9,57.9,5.3,269.0,4.2,4.5,4.1,0.651600
8,Duke University,62688,62688,0.0635,6570,0.9625,97800,0.9685,0.700444,Private Nonprofit,...,85.7,69.3,52.2,100.0,16.8,8693.0,4.4,4.8,4.4,0.729300
9,Emory University,57948,57948,0.1135,7017,0.9002,80137,0.9521,0.706744,Private Nonprofit,...,25.6,16.4,31.1,97.7,37.3,600.0,4.7,4.6,3.7,0.565966


In [11]:
# Save to file
merged_df.to_csv('merged_datasets.csv',index=False)